In [1]:
import sys
sys.path.append('..')
from MPRA_predict.utils import *

In [2]:
subprocess.run(
    ['python', 'predict_epi_features/0_predict_epi_feature.py', 
    '-i', 'data/Zhang_MPRA/Zhang_MPRA_final_200bp.csv', 
    '-o', 'predict_epi_features/outputs/Zhang_MPRA_final_200bp_Sei_pred.h5',
    '-m', 'Sei', 
    '-d', 'cuda:0'],
    cwd='..'
)

In [2]:
pred_array = load_h5('outputs/Zhang_MPRA_final_200bp_Sei_pred.h5')
print(pred_array.shape)

# pred_array = load_h5('outputs/Zhang_MPRA_final_Sei_pred.h5')
# print(pred_array.shape)

(17400, 21907)


In [3]:
MPRA_df = pd.read_csv("../data/Zhang_MPRA/Zhang_MPRA_final.csv")
MPRA_df

,seq,id,K562,HepG2,HEK293T,Jurkat
0,AAAAAAAAAAACCTCTGCCTTCATTGCAAATTCTTGGTGGTATGAA...,chr10:61837316-61837484,-2.061,1.465,-0.857,-2.474
1,AAAAAAAACAAAACAGGTAAGGATGAAAAGTTCCAGGTGAGAACAA...,chr12:45192846-45193014,-1.826,-0.830,-1.138,-1.781
2,AAAAAAAGCACAAAAATGCAAAAAACGTAGCACAGCAAAAAGGACA...,chr13:50241636-50241804,1.626,1.967,0.508,1.314
3,AAAAAAAGCATGGAGTTAGGAAAAATTTTCCTATGGGAGTGAGGTA...,chrX:102658676-102658844,-1.850,2.512,-0.919,-1.835
4,AAAAACACTTGAAAATTTTTGGCAGTCAGCAGGAATAGATAGGGAG...,chr7:79506606-79506774,-1.958,0.090,-1.128,-1.847
...,...,...,...,...,...,...
17395,TTTTTTCCCCTTTCCCTCCTCAGGTCTTCCTGTCAGCTGTTACTAT...,chr18:39701966-39702134,-1.720,2.907,-0.983,-1.588
17396,TTTTTTCTAATTGTTTTAGAGGTTGAAAGATACTGTGACTCTTCCG...,orig_backbone_change20%_18,-2.389,-1.626,-1.254,-2.107
17397,TTTTTTGTTGCTGTTCTGTTTTACCTTTCACCATGGAAAATTTGGT...,chr2:64128946-64129114,-2.233,0.682,-1.148,-2.925
17398,TTTTTTTCAGCTCACTTTTCCCTTGACTTCTAAAGCACATATTTAC...,chr5:99721596-99721764,-0.790,0.991,-1.126,-2.246


In [4]:
# cell_types = ['K562', 'HEK293T', 'HepG2', 'Jurkat']

cell_types = ['K562', 'HepG2', 'HEK293T', 'Jurkat']
cell_standard_names = ['K562_Leukemia_Cell', 'HepG2_Hepatocellular_Carcinoma', 'HEK293T_Epithelium_Embryonic_Kidney', 'Jurkat_T_Lymphocyte_Blood', 'HEK293_Epithelium_Embryonic_Kidney']

In [5]:
assays = ['DNase', 'H3K4me3', 'H3K27ac', 'CTCF']

In [32]:
df = pd.read_csv('../data/Sei/Sei_tracks_info.csv')
# df = df[df['AUROC'] > 0.95]
# df = df[df['AUROC'] > 0.9]
df_pivot = df.pivot_table(
    values="index", 
    index="cell_type", 
    columns="assay", 
    aggfunc=list,
)
df_pivot = df_pivot.map(lambda x: x if isinstance(x, list) else [])
df_pivot.shape

df_track = df_pivot.loc[cell_standard_names, assays]
df_track.map(len)

assay,DNase,H3K4me3,H3K27ac,CTCF
cell_type,,,,
K562_Leukemia_Cell,58,91,51,29
HepG2_Hepatocellular_Carcinoma,9,8,9,17
HEK293T_Epithelium_Embryonic_Kidney,1,10,1,0
Jurkat_T_Lymphocyte_Blood,3,18,9,2
HEK293_Epithelium_Embryonic_Kidney,0,17,5,12


In [33]:
df_track = pd.DataFrame(index=cell_types, columns=assays)
for i in range(len(cell_types)):
    cell_type = cell_types[i]
    cell_standard_name = cell_standard_names[i]
    df_track.loc[cell_type] = df_pivot.loc[cell_standard_name]

df_track.loc['HEK293T'] = df_pivot.loc['HEK293T_Epithelium_Embryonic_Kidney'] + df_pivot.loc['HEK293_Epithelium_Embryonic_Kidney']
df_track.map(len)

,DNase,H3K4me3,H3K27ac,CTCF
K562,58,91,51,29
HepG2,9,8,9,17
HEK293T,1,27,6,12
Jurkat,3,18,9,2


In [34]:
for i, cell_type in enumerate(cell_types):
    for j, assay in enumerate(assays):
        indice = df_track.loc[cell_type, assay]
        if len(indice) > 0:
            pred = logit(pred_array[:, indice], eps=1e-6).mean(1)
            MPRA_df[f'{cell_type}_{assay}'] = pred
        else:
            MPRA_df[f'{cell_type}_{assay}'] = np.nan

MPRA_df

,seq,id,K562,HepG2,HEK293T,Jurkat,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_CTCF,HepG2_DNase,HepG2_H3K4me3,HepG2_H3K27ac,HepG2_CTCF,HEK293T_DNase,HEK293T_H3K4me3,HEK293T_H3K27ac,HEK293T_CTCF,Jurkat_DNase,Jurkat_H3K4me3,Jurkat_H3K27ac,Jurkat_CTCF
0,AAAAAAAAAAACCTCTGCCTTCATTGCAAATTCTTGGTGGTATGAA...,chr10:61837316-61837484,-2.061,1.465,-0.857,-2.474,-3.111,-6.146,-5.816,-7.040,-1.237,-3.780,-4.959,-5.519,-5.998,-4.734,-5.699,-6.121,-5.763,-7.078,-7.784,-6.907
1,AAAAAAAACAAAACAGGTAAGGATGAAAAGTTCCAGGTGAGAACAA...,chr12:45192846-45193014,-1.826,-0.830,-1.138,-1.781,-6.212,-7.602,-7.796,-8.884,-4.615,-4.875,-5.767,-8.209,-7.159,-6.391,-7.054,-8.796,-6.683,-7.682,-9.228,-8.846
2,AAAAAAAGCACAAAAATGCAAAAAACGTAGCACAGCAAAAAGGACA...,chr13:50241636-50241804,1.626,1.967,0.508,1.314,-2.799,-4.548,-5.240,-5.564,-2.247,-3.026,-4.868,-4.893,-5.976,-4.287,-4.979,-5.087,-3.213,-5.103,-6.045,-5.169
3,AAAAAAAGCATGGAGTTAGGAAAAATTTTCCTATGGGAGTGAGGTA...,chrX:102658676-102658844,-1.850,2.512,-0.919,-1.835,-4.785,-7.349,-7.292,-7.466,-5.088,-5.927,-7.531,-6.773,-6.518,-5.772,-6.309,-6.700,-5.474,-7.449,-7.736,-6.735
4,AAAAACACTTGAAAATTTTTGGCAGTCAGCAGGAATAGATAGGGAG...,chr7:79506606-79506774,-1.958,0.090,-1.128,-1.847,0.077,-5.225,-5.118,-4.054,-1.136,-5.160,-6.164,-3.657,-5.225,-5.352,-5.326,-3.664,-2.389,-6.105,-6.633,-4.562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17395,TTTTTTCCCCTTTCCCTCCTCAGGTCTTCCTGTCAGCTGTTACTAT...,chr18:39701966-39702134,-1.720,2.907,-0.983,-1.588,-2.471,-4.798,-4.993,-6.350,-1.700,-3.620,-4.470,-4.889,-5.061,-3.647,-4.720,-4.985,-4.265,-5.967,-7.209,-6.198
17396,TTTTTTCTAATTGTTTTAGAGGTTGAAAGATACTGTGACTCTTCCG...,orig_backbone_change20%_18,-2.389,-1.626,-1.254,-2.107,-5.156,-7.778,-7.832,-7.776,-6.663,-7.319,-9.033,-7.888,-6.680,-6.237,-7.078,-7.709,-5.335,-8.052,-8.773,-7.413
17397,TTTTTTGTTGCTGTTCTGTTTTACCTTTCACCATGGAAAATTTGGT...,chr2:64128946-64129114,-2.233,0.682,-1.148,-2.925,-6.867,-8.351,-8.505,-9.099,-5.531,-7.079,-8.425,-8.311,-7.971,-7.353,-8.526,-8.586,-7.214,-9.307,-10.826,-8.633
17398,TTTTTTTCAGCTCACTTTTCCCTTGACTTCTAAAGCACATATTTAC...,chr5:99721596-99721764,-0.790,0.991,-1.126,-2.246,-3.077,-6.771,-6.363,-6.921,-1.759,-5.685,-6.114,-5.442,-5.839,-5.648,-6.401,-6.349,-5.439,-8.303,-9.115,-6.969


In [35]:
for j, assay in enumerate(assays):
    pearson_df = pd.DataFrame()
    for cell_type_0 in cell_types:
        for i, cell_type in enumerate(cell_types):
            pred = MPRA_df[f'{cell_type}_{assay}']
            true = MPRA_df[f'{cell_type_0}']
            r, _ = pearson(pred, true)
            pearson_df.loc[f'{cell_type}_{assay}', f'{cell_type_0}'] = r
    print(pearson_df)

               K562  HepG2  HEK293T  Jurkat
K562_DNase    0.612  0.418    0.510   0.550
HepG2_DNase   0.437  0.526    0.330   0.298
HEK293T_DNase 0.393  0.141    0.308   0.421
Jurkat_DNase  0.445  0.204    0.370   0.552
                 K562  HepG2  HEK293T  Jurkat
K562_H3K4me3    0.561  0.394    0.431   0.528
HepG2_H3K4me3   0.487  0.508    0.364   0.409
HEK293T_H3K4me3 0.230  0.060    0.199   0.352
Jurkat_H3K4me3  0.428  0.192    0.317   0.551
                 K562  HepG2  HEK293T  Jurkat
K562_H3K27ac    0.539  0.391    0.436   0.503
HepG2_H3K27ac   0.390  0.524    0.294   0.277
HEK293T_H3K27ac 0.287  0.185    0.272   0.332
Jurkat_H3K27ac  0.399  0.170    0.295   0.516
              K562  HepG2  HEK293T  Jurkat
K562_CTCF    0.444  0.276    0.289   0.344
HepG2_CTCF   0.397  0.311    0.247   0.275
HEK293T_CTCF 0.431  0.247    0.311   0.373
Jurkat_CTCF  0.461  0.268    0.310   0.413


In [ ]:
# after remove nan, len(x) < 2, pearson = nan
# after remove nan, len(x) < 2, pearson = nan
# after remove nan, len(x) < 2, pearson = nan
# after remove nan, len(x) < 2, pearson = nan
#                K562  HepG2  HEK293T  Jurkat
# K562_DNase    0.612  0.418    0.510   0.550
# HepG2_DNase   0.437  0.526    0.330   0.298
# HEK293T_DNase   NaN    NaN      NaN     NaN
# Jurkat_DNase  0.445  0.204    0.370   0.552
#                  K562  HepG2  HEK293T  Jurkat
# K562_H3K4me3    0.560  0.393    0.430   0.527
# HepG2_H3K4me3   0.487  0.508    0.364   0.409
# HEK293T_H3K4me3 0.211  0.041    0.186   0.340
# Jurkat_H3K4me3  0.428  0.192    0.317   0.551
#                  K562  HepG2  HEK293T  Jurkat
# K562_H3K27ac    0.566  0.427    0.449   0.515
# HepG2_H3K27ac   0.384  0.524    0.291   0.270
# HEK293T_H3K27ac 0.252  0.154    0.245   0.306
# Jurkat_H3K27ac  0.399  0.170    0.295   0.516
#               K562  HepG2  HEK293T  Jurkat
# K562_CTCF    0.444  0.276    0.289   0.344
# HepG2_CTCF   0.397  0.311    0.247   0.275
# HEK293T_CTCF 0.431  0.247    0.311   0.373
# Jurkat_CTCF  0.461  0.268    0.310   0.413

In [36]:
MPRA_df.to_csv('../data/Zhang_MPRA/Zhang_MPRA_Sei_feature.csv', index=False)

In [37]:
MPRA_df = pd.read_csv('../data/Zhang_MPRA/Zhang_MPRA_Sei_feature.csv')
MPRA_df

,seq,id,K562,HepG2,HEK293T,Jurkat,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_CTCF,HepG2_DNase,HepG2_H3K4me3,HepG2_H3K27ac,HepG2_CTCF,HEK293T_DNase,HEK293T_H3K4me3,HEK293T_H3K27ac,HEK293T_CTCF,Jurkat_DNase,Jurkat_H3K4me3,Jurkat_H3K27ac,Jurkat_CTCF
0,AAAAAAAAAAACCTCTGCCTTCATTGCAAATTCTTGGTGGTATGAA...,chr10:61837316-61837484,-2.061,1.465,-0.857,-2.474,-3.111,-6.146,-5.816,-7.040,-1.237,-3.780,-4.959,-5.519,-5.998,-4.734,-5.699,-6.121,-5.763,-7.078,-7.784,-6.907
1,AAAAAAAACAAAACAGGTAAGGATGAAAAGTTCCAGGTGAGAACAA...,chr12:45192846-45193014,-1.826,-0.830,-1.138,-1.781,-6.212,-7.602,-7.796,-8.884,-4.615,-4.875,-5.767,-8.209,-7.159,-6.391,-7.054,-8.796,-6.683,-7.682,-9.228,-8.846
2,AAAAAAAGCACAAAAATGCAAAAAACGTAGCACAGCAAAAAGGACA...,chr13:50241636-50241804,1.626,1.967,0.508,1.314,-2.799,-4.548,-5.240,-5.564,-2.247,-3.026,-4.868,-4.893,-5.976,-4.287,-4.979,-5.087,-3.213,-5.103,-6.045,-5.169
3,AAAAAAAGCATGGAGTTAGGAAAAATTTTCCTATGGGAGTGAGGTA...,chrX:102658676-102658844,-1.850,2.512,-0.919,-1.835,-4.785,-7.349,-7.292,-7.466,-5.088,-5.927,-7.531,-6.773,-6.518,-5.772,-6.309,-6.700,-5.474,-7.449,-7.736,-6.735
4,AAAAACACTTGAAAATTTTTGGCAGTCAGCAGGAATAGATAGGGAG...,chr7:79506606-79506774,-1.958,0.090,-1.128,-1.847,0.077,-5.225,-5.118,-4.054,-1.136,-5.160,-6.164,-3.657,-5.225,-5.352,-5.326,-3.664,-2.389,-6.105,-6.633,-4.562
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17395,TTTTTTCCCCTTTCCCTCCTCAGGTCTTCCTGTCAGCTGTTACTAT...,chr18:39701966-39702134,-1.720,2.907,-0.983,-1.588,-2.471,-4.798,-4.993,-6.350,-1.700,-3.620,-4.470,-4.889,-5.061,-3.647,-4.720,-4.985,-4.265,-5.967,-7.209,-6.198
17396,TTTTTTCTAATTGTTTTAGAGGTTGAAAGATACTGTGACTCTTCCG...,orig_backbone_change20%_18,-2.389,-1.626,-1.254,-2.107,-5.156,-7.778,-7.832,-7.776,-6.663,-7.319,-9.033,-7.888,-6.680,-6.237,-7.078,-7.709,-5.335,-8.052,-8.773,-7.413
17397,TTTTTTGTTGCTGTTCTGTTTTACCTTTCACCATGGAAAATTTGGT...,chr2:64128946-64129114,-2.233,0.682,-1.148,-2.925,-6.867,-8.351,-8.505,-9.099,-5.531,-7.079,-8.425,-8.311,-7.971,-7.353,-8.526,-8.586,-7.214,-9.307,-10.826,-8.633
17398,TTTTTTTCAGCTCACTTTTCCCTTGACTTCTAAAGCACATATTTAC...,chr5:99721596-99721764,-0.790,0.991,-1.126,-2.246,-3.077,-6.771,-6.363,-6.921,-1.759,-5.685,-6.114,-5.442,-5.839,-5.648,-6.401,-6.349,-5.439,-8.303,-9.115,-6.969


In [38]:
cols = cell_types + [f'{cell_type}_{assay}' for cell_type in cell_types for assay in assays]

x = MPRA_df[cols].copy()
x_mean = x.mean(axis=0)
x_std = x.std(axis=0)
x = (x - x_mean) / x_std

MPRA_df[cols] = x
MPRA_df

,seq,id,K562,HepG2,HEK293T,Jurkat,K562_DNase,K562_H3K4me3,K562_H3K27ac,K562_CTCF,HepG2_DNase,HepG2_H3K4me3,HepG2_H3K27ac,HepG2_CTCF,HEK293T_DNase,HEK293T_H3K4me3,HEK293T_H3K27ac,HEK293T_CTCF,Jurkat_DNase,Jurkat_H3K4me3,Jurkat_H3K27ac,Jurkat_CTCF
0,AAAAAAAAAAACCTCTGCCTTCATTGCAAATTCTTGGTGGTATGAA...,chr10:61837316-61837484,-0.325,2.187,-0.159,-1.084,0.048,0.380,0.873,-0.283,2.048,1.585,2.216,0.277,-0.275,0.233,0.408,-0.254,-1.616,-0.792,-0.666,-0.615
1,AAAAAAAACAAAACAGGTAAGGATGAAAAGTTCCAGGTGAGAACAA...,chr12:45192846-45193014,-0.185,0.146,-0.491,-0.586,-1.655,-0.760,-0.955,-0.989,0.384,0.907,1.675,-0.668,-1.446,-1.196,-0.881,-1.467,-2.040,-1.196,-1.660,-1.341
2,AAAAAAAGCACAAAAATGCAAAAAACGTAGCACAGCAAAAAGGACA...,chr13:50241636-50241804,1.867,2.633,1.457,1.643,0.219,1.630,1.404,0.282,1.550,2.051,2.277,0.497,-0.253,0.618,1.093,0.214,-0.441,0.529,0.530,0.036
3,AAAAAAAGCATGGAGTTAGGAAAAATTTTCCTATGGGAGTGAGGTA...,chrX:102658676-102658844,-0.199,3.117,-0.232,-0.625,-0.871,-0.562,-0.489,-0.446,0.150,0.256,0.494,-0.163,-0.800,-0.663,-0.172,-0.517,-1.483,-1.040,-0.634,-0.551
4,AAAAACACTTGAAAATTTTTGGCAGTCAGCAGGAATAGATAGGGAG...,chr7:79506606-79506774,-0.263,0.964,-0.479,-0.633,1.799,1.100,1.517,0.861,2.098,0.731,1.409,0.931,0.504,-0.301,0.763,0.859,-0.061,-0.142,0.125,0.263
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17395,TTTTTTCCCCTTTCCCTCCTCAGGTCTTCCTGTCAGCTGTTACTAT...,chr18:39701966-39702134,-0.122,3.469,-0.308,-0.447,0.400,1.435,1.632,-0.019,1.820,1.684,2.543,0.498,0.668,1.170,1.339,0.260,-0.926,-0.049,-0.271,-0.350
17396,TTTTTTCTAATTGTTTTAGAGGTTGAAAGATACTGTGACTCTTCCG...,orig_backbone_change20%_18,-0.519,-0.561,-0.629,-0.821,-1.075,-0.897,-0.988,-0.564,-0.626,-0.605,-0.512,-0.555,-0.963,-1.064,-0.903,-0.974,-1.419,-1.443,-1.347,-0.805
17397,TTTTTTGTTGCTGTTCTGTTTTACCTTTCACCATGGAAAATTTGGT...,chr2:64128946-64129114,-0.427,1.491,-0.503,-1.409,-2.014,-1.345,-1.609,-1.071,-0.068,-0.457,-0.105,-0.704,-2.264,-2.026,-2.281,-1.371,-2.285,-2.283,-2.760,-1.261
17398,TTTTTTTCAGCTCACTTTTCCCTTGACTTCTAAAGCACATATTTAC...,chr5:99721596-99721764,0.431,1.765,-0.476,-0.921,0.067,-0.109,0.368,-0.237,1.791,0.405,1.443,0.304,-0.115,-0.556,-0.259,-0.358,-1.466,-1.612,-1.582,-0.638


In [39]:
MPRA_df.to_csv('../data/Zhang_MPRA/Zhang_MPRA_Sei_feature_zscore.csv', index=False)